In [8]:
from utils.connect import connect
# from utils.aed_rows import add_row, edit_row, delete_row
import pandas as pd
from utils.schema import Data
from pandantic import Pandantic

In [9]:
# Get data as Pandas DataFrame
sheet = connect()

In [10]:
# Get raw values including headers
raw_data = sheet.get_values()
headers = raw_data[0]
values = raw_data[1:]

# Create DataFrame with proper number parsing
df = pd.DataFrame(values, columns=headers)

# Convert Amount and Total with proper handling of decimal separators
df['Amount'] = df['Amount'].apply(lambda x: float(str(x).replace('.', '').replace(',', '.')))
df['Total'] = df['Total'].apply(lambda x: float(str(x).replace('.', '').replace(',', '.')))
df['Type'] = df['Type'].astype(int)

In [11]:
df.Total.head()

0    5960.5
1    5963.0
2    6003.0
3    6027.0
4    6036.5
Name: Total, dtype: float64

In [6]:
# Convert the sheet to a DataFrame
df = pd.DataFrame(sheet.get_all_records())
df

,Date,Category,Amount,Total,Type
0,2025-02-13,Fun,250.0,5.9605,0
1,2025-02-13,Fuel,2400.0,6.0030,0
2,2025-02-13,Mprikis,4000.0,5.9630,0
3,2025-02-12,Fun,950.0,6.0270,0
4,2025-02-12,Mprikis,1100.0,6.0365,0
...,...,...,...,...,...
2445,2022-02-22,Gifts,900.0,31.6308,1
2446,2022-02-21,Fun,250.0,31.6218,0
2447,2022-02-21,Fun,300.0,31.6243,0
2448,2022-02-21,Fuel,450.0,31.6273,0


In [7]:
# Convert 'Date' to datetime format for sorting
df["Date"] = pd.to_datetime(df["Date"])

# Sort by Date (latest first)
df = df.sort_values(by="Date", ascending=False).reset_index(drop=True)

# Convert "Total" column to numeric (handling empty strings)
df["Total"] = pd.to_numeric(df["Total"], errors="coerce")

df["Amount"] = df['Amount'].apply(lambda x: x * 1000 if x < 10 else x/100)
df["Total"] = df["Total"]*1000


In [8]:
df

,Date,Category,Amount,Total,Type
0,2025-02-13,Fun,2.5,5960.5,0
1,2025-02-13,Mprikis,40.0,5963.0,0
2,2025-02-13,Fuel,24.0,6003.0,0
3,2025-02-12,Fun,9.5,6027.0,0
4,2025-02-12,Mprikis,11.0,6036.5,0
...,...,...,...,...,...
2445,2022-02-22,Gifts,9.0,31630.8,1
2446,2022-02-21,Fun,2.5,31621.8,0
2447,2022-02-21,Fun,3.0,31624.3,0
2448,2022-02-21,Fuel,4.5,31627.3,0


In [9]:
# Iterate and compute the Total column
for i in range(1, len(df)):  # Start from second row
    if df.loc[i-1, "Type"] == 0:  # Expense
        df.loc[i, "Total"] = df.loc[i-1, "Total"] + df.loc[i-1, "Amount"]  
    else:  # Income (Type == 1)
        df.loc[i, "Total"] = df.loc[i-1, "Total"] - df.loc[i-1, "Amount"]  

# Round the Total column to 2 decimal places
df["Total"] = round(df["Total"], 2)

In [10]:
display(df)

,Date,Category,Amount,Total,Type
0,2025-02-13,Fun,2.5,5960.5,0
1,2025-02-13,Mprikis,40.0,5963.0,0
2,2025-02-13,Fuel,24.0,6003.0,0
3,2025-02-12,Fun,9.5,6027.0,0
4,2025-02-12,Mprikis,11.0,6036.5,0
...,...,...,...,...,...
2445,2022-02-22,Gifts,9.0,11349.8,1
2446,2022-02-21,Fun,2.5,11340.8,0
2447,2022-02-21,Fun,3.0,11343.3,0
2448,2022-02-21,Fuel,4.5,11346.3,0


In [11]:
# Convert DataFrame rows to Data objects
validated_data = [Data(
    Date=row['Date'],
    Category=row['Category'],
    Amount=row['Amount'],
    Total=row['Total'],
    Type=row['Type']
) for index, row in df.iterrows()]

In [12]:
def upload_validated_data(validated_data, sheet):
    # Convert list of Data objects to list of lists
    rows = []
    
    # Add headers first
    headers = ['Date', 'Category', 'Amount', 'Total', 'Type']
    rows.append(headers)
    
    # Convert each Data object to a list
    for data in validated_data:
        row = [
            data.Date.strftime('%Y-%m-%d'),  # Format datetime to string
            data.Category,
            data.Amount,
            data.Total,
            data.Type
        ]
        rows.append(row)
    
    try:
        # Clear existing data (optional)
        sheet.clear()
        
        # Update the sheet with all data at once
        sheet.update(rows)
        print(f"Successfully uploaded {len(validated_data)} rows to Google Sheets")
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")

# Usage
sheet = connect()
upload_validated_data(validated_data, sheet)

Successfully uploaded 2450 rows to Google Sheets


In [7]:
validated_data

[Data(Date=Timestamp('2025-02-13 00:00:00'), Category=<Categories.Fuel: 'Fuel'>, Amount=24.0, Total=5936.5, Type=0),
 Data(Date=Timestamp('2025-02-13 00:00:00'), Category=<Categories.Mprikis: 'Mprikis'>, Amount=40.0, Total=5976.5, Type=0),
 Data(Date=Timestamp('2025-02-13 00:00:00'), Category=<Categories.Fun: 'Fun'>, Amount=2.5, Total=5979.0, Type=0),
 Data(Date=Timestamp('2025-02-12 00:00:00'), Category=<Categories.Mprikis: 'Mprikis'>, Amount=11.0, Total=6003.0, Type=0),
 Data(Date=Timestamp('2025-02-12 00:00:00'), Category=<Categories.Fun: 'Fun'>, Amount=9.5, Total=6012.5, Type=0),
 Data(Date=Timestamp('2025-02-12 00:00:00'), Category=<Categories.Education: 'Education'>, Amount=15.0, Total=6027.5, Type=0),
 Data(Date=Timestamp('2025-02-12 00:00:00'), Category=<Categories.Supermarket: 'Supermarket'>, Amount=18.0, Total=6045.5, Type=0),
 Data(Date=Timestamp('2025-02-11 00:00:00'), Category=<Categories.Dad: 'Dad'>, Amount=70.0, Total=5975.5, Type=1),
 Data(Date=Timestamp('2025-02-11 00:

In [12]:
# # Convert Date column to datetime
# df['Date'] = pd.to_datetime(df['Date'])

# # Convert Total column to float, replacing empty strings with NaN first
# df['Total'] = (df['Total']
#                .str.replace('$', '')
#                .str.replace(',', '')
#                .replace('', pd.NA)  # Replace empty strings with NaN
#                .astype(float))

# # If you want to fill NaN values with 0 or another value, you can do:
# df['Total'] = df['Total'].fillna(0)

# Verify the datatypes
print(df.dtypes)

Date        datetime64[ns]
Category            object
Amount             float64
Total              float64
Type                 int64
dtype: object


In [21]:
df.Category.unique()

array(['Mprikis', 'Fub', 'Fun', 'Education', 'Supermarket', 'Dad', 'Gym',
       'Other', 'Vodafone', 'Fuel', 'Electricity', 'Subscriptions',
       'Parking', 'Rent', 'Haircut', 'Family', 'Job', 'Me', 'Mom',
       'other', 'Health', 'Hobies', 'Building fees', 'Gifts', 'Allowance',
       'Lena', 'Vacation', 'Water', 'MOTOmaintenance/oil/insurance',
       'Accountant', 'Private', 'Efka', 'Gift'], dtype=object)

In [ ]:
# Check the dataframe schema
validator = Pandantic(schema=Data)
df_valid = validator.validate(dataframe=df, errors="raise")
df_valid

ValidationError: 1 validation error for Data
Total
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/float_parsing

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2449 entries, 0 to 2448
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      2449 non-null   object 
 1   Category  2449 non-null   object 
 2   Amount    2449 non-null   float64
 3   Total     2449 non-null   object 
 4   Type      2449 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 95.8+ KB


In [ ]:
# New row to add
new_row = ["2025-02-02", "fun", 2, "", 0]  # NaN is for Total

# Insert row
sheet.insert_row(new_row, 2)

print("Row added successfully!")


Row added successfully!


In [14]:
# Iterate and compute the Total column
for i in range(1, len(df)):  # Start from second row
    if df.loc[i, "Total"] == "" or pd.isna(df.loc[i, "Total"]):  # If Total is empty
        if df.loc[i, "Type"] == 0:  # Expense
            df.loc[i, "Total"] = df.loc[i-1, "Total"] + df.loc[i, "Amount"]
        else:  # Income
            df.loc[i, "Total"] = df.loc[i-1, "Total"] - df.loc[i, "Amount"]

# Convert Total column back to integer
df["Total"] = df["Total"].astype(float)

# Display the result
# import ace_tools as tools
# tools.display_dataframe_to_user(name="Updated Total Calculation", dataframe=df)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2449 entries, 0 to 2448
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      2449 non-null   datetime64[ns]
 1   Category  2449 non-null   object        
 2   Amount    2449 non-null   float64       
 3   Total     2447 non-null   float64       
 4   Type      2449 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 95.8+ KB


In [20]:
# Convert 'Date' to datetime format (if not already converted)
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# Now convert back to string format for Google Sheets
df["Date"] = df["Date"].dt.strftime("%Y-%m-%d")

# Convert DataFrame to list of lists (Google Sheets format)
data_to_push = [df.columns.tolist()] + df.values.tolist()

# Overwrite entire sheet with updated data
sh.update("A1", data_to_push)

print("✅ Data successfully updated in Google Sheets!")

/tmp/ipykernel_28458/3795469456.py:11: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sh.update("A1", data_to_push)


✅ Data successfully updated in Google Sheets!


In [22]:
# Open the Google Sheet
sheet = sh

# ✅ 2. Read Data from Google Sheets into a DataFrame
data = sheet.get_all_records()
df = pd.DataFrame(data)

# ✅ 3. Convert 'Date' to datetime format (handling any errors)
df["Date"] = pd.to_datetime(df["Date"], errors="coerce")

# ✅ 4. Sort DataFrame by Date (latest first)
df = df.sort_values(by="Date", ascending=False).reset_index(drop=True)

# ✅ 5. Convert "Total" column to numeric (handling empty values)
df["Total"] = pd.to_numeric(df["Total"], errors="coerce")

# ✅ 6. Shift "Total" column down by one row
df["Total"] = df["Total"].shift(1)

# ✅ 7. Set the first row's "Total" to an empty string
df.loc[0, "Total"] = ""

# ✅ 8. Ensure "Total" remains integer where possible
df["Total"] = pd.to_numeric(df["Total"], errors="coerce").astype("Int64")  # Keeps NaN as blank instead of float NaN

# ✅ 9. Convert Date to string format before pushing
df["Date"] = df["Date"].dt.strftime("%Y-%m-%d")

# ✅ 10. Push updated DataFrame back to Google Sheets
data_to_push = [df.columns.tolist()] + df.astype(str).values.tolist()  # Ensure all values are strings for Google Sheets

# Overwrite entire sheet with updated data
sheet.update("A1", data_to_push)

print("✅ Data successfully updated in Google Sheets!")


/tmp/ipykernel_28458/2312491031.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[0, "Total"] = ""
/tmp/ipykernel_28458/2312491031.py:33: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update("A1", data_to_push)


✅ Data successfully updated in Google Sheets!
